In [1]:
import numpy as np

In [2]:
import pandas as pd

In [3]:
credits = pd.read_csv('tmdb_5000_credits.csv')
movies = pd.read_csv('tmdb_5000_movies.csv')

In [4]:
movies = movies.merge(credits,on='title')

In [5]:
#genre
#id
#keywords
#title
#overview
#cast
#crew
movies = movies[['movie_id','title','overview','genres','keywords','cast','crew']]

In [6]:
movies.dropna(inplace=True)

In [7]:
movies.isnull().sum()

movie_id    0
title       0
overview    0
genres      0
keywords    0
cast        0
crew        0
dtype: int64

In [8]:
movies.duplicated().sum()

np.int64(0)

In [9]:
import ast

In [10]:
#converting the genre dictionary to a list
def convert(obj):
    l=[]
    for i in ast.literal_eval(obj):
        l.append(i['name'])
    return l

In [11]:
movies['genres']=movies['genres'].apply(convert)

In [12]:
movies['keywords']=movies['keywords'].apply(convert)

In [13]:
#conversion for movie cast name
def convert3(obj):
    l=[]
    c=0
    for i in ast.literal_eval(obj):
        if c!=3:
            l.append(i['name'])
            c+=1
        else:
            break
    return l

In [14]:
movies['cast']=movies['cast'].apply(convert3)

In [15]:
def fetchdirectorname(obj):
    l=[]
    for i in ast.literal_eval(obj):
        if i['job']=='Director':
            l.append(i['name'])
            break
    return l

In [16]:
movies['crew']=movies['crew'].apply(fetchdirectorname)

In [17]:
movies['overview']=movies['overview'].apply(lambda x:x.split())

In [18]:
movies['genres']=movies['genres'].apply(lambda x:[i.replace(" ","") for i in x])

movies['keywords']=movies['keywords'].apply(lambda x:[i.replace(" ","") for i in x])

movies['cast']=movies['cast'].apply(lambda x:[i.replace(" ","") for i in x])

movies['crew']=movies['crew'].apply(lambda x:[i.replace(" ","") for i in x])

In [19]:
movies['tags']=movies['overview']+movies['genres']+movies['keywords']+movies['cast']+movies['crew']

In [20]:
new_mov=movies[['movie_id','title','tags']]

In [21]:
new_mov['tags']=new_mov['tags'].apply(lambda x: " ".join(x))

C:\Users\kekad\AppData\Local\Temp\ipykernel_16836\2617692074.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_mov['tags']=new_mov['tags'].apply(lambda x: " ".join(x))


In [22]:
new_mov['tags']=new_mov['tags'].apply(lambda x:x.lower())

C:\Users\kekad\AppData\Local\Temp\ipykernel_16836\3266829716.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_mov['tags']=new_mov['tags'].apply(lambda x:x.lower())


In [23]:
import nltk
from nltk.stem.porter import PorterStemmer
ps=PorterStemmer()

In [24]:
def stem(text):
    n=[]
    for i in text.split():
        n.append(ps.stem(i))
    return " ".join(n)

In [25]:
new_mov['tags']=new_mov['tags'].apply(stem)

C:\Users\kekad\AppData\Local\Temp\ipykernel_16836\2633675320.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_mov['tags']=new_mov['tags'].apply(stem)


In [26]:
from sklearn.feature_extraction.text import CountVectorizer

In [27]:
cv=CountVectorizer(max_features=5000,stop_words='english')

In [28]:
vectors=cv.fit_transform(new_mov['tags']).toarray()

In [29]:
cv.get_feature_names_out()

array(['000', '007', '10', ..., 'zone', 'zoo', 'zooeydeschanel'],
      shape=(5000,), dtype=object)

In [30]:
from sklearn.metrics.pairwise import cosine_similarity

In [31]:
similarity=cosine_similarity(vectors)

In [32]:
sorted(list(enumerate(similarity[0])),reverse=True,key=lambda x:x[1])[1:6]

[(1214, np.float64(0.28676966733820225)),
 (2405, np.float64(0.26901379342448517)),
 (3728, np.float64(0.2605130246476754)),
 (507, np.float64(0.255608593705383)),
 (539, np.float64(0.25038669783359574))]

In [33]:
#main recommendation logic function
def recommend(movie):
    movie_index = new_mov[new_mov['title'] == movie].index[0]
    distances = similarity[movie_index]
    movies_list = sorted(list(enumerate(distances)),reverse=True,key=lambda x:x[1])[1:6]

    for i in movies_list:
        print(new_mov.iloc[i[0]].title)

In [34]:
recommend('Avatar')

Aliens vs Predator: Requiem
Aliens
Falcon Rising
Independence Day
Titan A.E.


In [39]:
import pickle
pickle.dump(new_mov.to_dict,open('moviesdict.pkl','wb'))
pickle.dump(similarity,open('similarity.pkl','wb'))

In [ ]:
new_mov.to_dict() 